In [1]:
import numpy as np
import xarray as xr
import glob
import os

In [2]:
input_files = glob.glob("data/SEAS5/*.nc")
output_dir = "data/SEAS5_subset"
os.makedirs(output_dir, exist_ok=True)
input_files

['data/SEAS5\\ecmwf_year-1993_month-01_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-02_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-03_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-04_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-05_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-06_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-07_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-08_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-09_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-10_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-11_sfc.nc',
 'data/SEAS5\\ecmwf_year-1993_month-12_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-01_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-02_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-03_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-04_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-05_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-06_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-07_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-08_sfc.nc',
 'data/SEAS5\\ecmwf_year-1994_month-09_s

In [3]:
# Bounding box
lon_min, lon_max, lat_min, lat_max = -105.0, -89.0, 36.0, 45.0

# Variable you want to extract
var_names = ["T2MAX", "T2MIN"]

# Output folder
output_dir = "data/SEAS5_subset"
os.makedirs(output_dir, exist_ok=True)

for var_name in var_names:
    for f in input_files:
        print(f"Processing {f} ...")
        ds = xr.open_dataset(f)

        ds = ds[[var_name, "lat", "lon"]]

        # Build bounding box mask
        lon = ds["lon"]
        lat = ds["lat"]
        mask = (
            (lon >= lon_min) & (lon <= lon_max) &
            (lat >= lat_min) & (lat <= lat_max)
        ).compute()

        # Apply mask
        ds_subset = ds.where(mask, drop=True)
        ds_subset = ds_subset.set_coords(["lon", "lat"])

        # Build output filename with variable name
        base = os.path.splitext(os.path.basename(f))[0]
        out_path = os.path.join(output_dir, f"{base}_{var_name}_subset.nc")

        # Save
        ds_subset.to_netcdf(out_path)
        
ds_subset

Processing data/SEAS5\ecmwf_year-1993_month-01_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-02_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-03_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-04_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-05_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-06_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-07_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-08_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-09_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-10_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-11_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1993_month-12_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1994_month-01_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1994_month-02_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1994_month-03_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1994_month-04_sfc.nc ...
Processing data/SEAS5\ecmwf_year-1994_month-05_sfc.nc ...
Processing dat

<xarray.Dataset> Size: 2MB
Dimensions:  (number: 51, time: 6, south_north: 33, west_east: 47)
Coordinates:
    lat      (south_north, west_east) float32 6kB nan nan nan ... nan nan nan
    lon      (south_north, west_east) float32 6kB nan nan nan ... nan nan nan
  * number   (number) int64 408B 0 1 2 3 4 5 6 7 8 ... 43 44 45 46 47 48 49 50
  * time     (time) datetime64[ns] 48B 2025-07-01 2025-08-01 ... 2025-12-01
Dimensions without coordinates: south_north, west_east
Data variables:
    T2MIN    (number, time, south_north, west_east) float32 2MB nan nan ... nan
Attributes:
    regrid_method:  bilinear